In [ ]:
import pandas as pd
import os
import re
import concurrent.futures
from openai import OpenAI
from tqdm import tqdm
import time

# ================= 配置区域 =================

# 1. 大模型配置 (阿里云 DashScope)
LLM_API_KEY = "sk-fa13f585000140deabdfa506b25a7f3d"
# 阿里云 OpenAI 兼容接口地址
LLM_API_BASE = "https://dashscope.aliyuncs.com/compatible-mode/v1"
MODEL_NAME = "qwen3-30b-a3b-instruct-2507" # 或者是 qwen-plus / qwen-max，请确保您有该模型权限

# 2. 并发数量
# 云端接口通常有速率限制(RPM/TPM)，建议不要设置过大，5-8 比较稳妥
MAX_WORKERS = 5

# 3. 文件路径
try:
    CURRENT_DIR = os.path.dirname(os.path.abspath(__file__))
except NameError:
    CURRENT_DIR = os.getcwd()

# 输入文件
INPUT_FILENAME = "api_retrieval_split_result.xlsx"
INPUT_PATH = os.path.join(CURRENT_DIR, INPUT_FILENAME)

# 输出文件
OUTPUT_PATH = os.path.join(CURRENT_DIR, "final_analysis_comparison_aliyun.xlsx")

# ===========================================

# 初始化客户端
client = OpenAI(api_key=LLM_API_KEY, base_url=LLM_API_BASE)

def clean_model_output(text):
    """清洗模型输出"""
    if not text: return ""
    text = re.sub(r'<think>.*?</think>', '', text, flags=re.DOTALL)
    text = text.replace('```', '').strip()
    return text

def merge_columns(row, prefix, count):
    """合并多个召回结果列"""
    content_list = []
    for i in range(1, count + 1):
        col_name = f"{prefix}{i}"
        if col_name not in row and prefix == "召回结果":
             col_name = f"召回结果{i}"

        if col_name in row and pd.notna(row[col_name]):
            val = str(row[col_name]).strip()
            if val and val not in content_list:
                content_list.append(f"[{i}]: {val}")
    
    return "\n".join(content_list)[:2500]

def call_llm_with_retry(prompt, retries=3):
    """带重试机制的LLM调用"""
    for i in range(retries):
        try:
            chat_response = client.chat.completions.create(
                model=MODEL_NAME,
                messages=[
                    {"role": "system", "content": "你是一个精准的数据分析师。请直接输出分析结果，严禁包含思考过程(<think>)，严禁输出客套话。字数严格控制在100字以内。"},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.1,
                max_tokens=256
            )
            return clean_model_output(chat_response.choices[0].message.content)
        except Exception as e:
            if i == retries - 1:
                return f"调用失败: {str(e)}"
            time.sleep(1) # 失败等待1秒重试
    return "调用失败"

def analyze_single_row(row):
    """处理单行数据"""
    try:
        question = str(row.get('问题', row.get('原问题', '')))
        
        baseline_content = merge_columns(row, "召回结果", 5)
        scheme1_content = merge_columns(row, "方案1_召回结果", 3)
        scheme2_content = merge_columns(row, "方案2_召回结果", 3)

        if not baseline_content: baseline_content = "（无召回内容）"
        if not scheme1_content: scheme1_content = "（无召回内容）"
        if not scheme2_content: scheme2_content = "（无召回内容）"

        # Prompt 1: 方案1 评估
        p1 = f"【问题】：{question}\n【检索内容】：{scheme1_content}\n请一句话评估：上述检索内容是否包含回答问题所需的关键事实？缺失了什么？(限100字)"
        
        # Prompt 2: 方案2 评估
        p2 = f"【问题】：{question}\n【检索内容】：{scheme2_content}\n请一句话评估：上述检索内容是否包含回答问题所需的关键事实？缺失了什么？(限100字)"

        # Prompt 3: 方案1 vs 原召回
        p3 = f"【问题】：{question}\n【原检索】：{baseline_content}\n【新检索(长难句扩写)】：{scheme1_content}\n请对比：新检索相比原检索，准确性是提升还是下降？主要体现在哪？(限100字，直接给结论)"

        # Prompt 4: 方案2 vs 原召回
        p4 = f"【问题】：{question}\n【原检索】：{baseline_content}\n【新检索(潜在追问)】：{scheme2_content}\n请对比：新检索相比原检索，信息丰富度是提升还是下降？是否引入了噪音？(限100字，直接给结论)"

        # 串行调用4次
        r1 = call_llm_with_retry(p1)
        r2 = call_llm_with_retry(p2)
        r3 = call_llm_with_retry(p3)
        r4 = call_llm_with_retry(p4)

        return r1, r2, r3, r4

    except Exception as e:
        err_msg = f"处理出错: {str(e)}"
        return err_msg, err_msg, err_msg, err_msg

def main():
    print(f"正在读取文件: {INPUT_PATH}")
    if not os.path.exists(INPUT_PATH):
        print("❌ 文件不存在！")
        return

    df = pd.read_excel(INPUT_PATH)
    total_rows = len(df)
    print(f"共 {total_rows} 条数据，使用 {MAX_WORKERS} 线程并发处理...")

    results_list = []

    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        rows = df.to_dict('records')
        results_iterator = list(tqdm(executor.map(analyze_single_row, rows), total=total_rows, desc="阿里云LLM分析中"))
        results_list = results_iterator

    col1, col2, col3, col4 = [], [], [], []
    for res in results_list:
        col1.append(res[0])
        col2.append(res[1])
        col3.append(res[2])
        col4.append(res[3])

    df["方案1_效果评估"] = col1
    df["方案2_效果评估"] = col2
    df["方案1_vs_原召回_对比"] = col3
    df["方案2_vs_原召回_对比"] = col4

    print("分析完成，正在保存...")
    try:
        df.to_excel(OUTPUT_PATH, index=False)
        print(f"🎉 处理成功！结果保存在: {OUTPUT_PATH}")
    except PermissionError:
        print("❌ 保存失败：请关闭 Excel 文件后重试。")

if __name__ == "__main__":
    main()

正在读取文件: d:\jupyter\api_retrieval_split_result.xlsx
共 83 条数据，使用 5 线程并发处理...


阿里云LLM分析中: 100%|██████████| 83/83 [00:51<00:00,  1.63it/s]

分析完成，正在保存...
🎉 处理成功！结果保存在: d:\jupyter\final_analysis_comparison_aliyun.xlsx
